<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/Conversion_PyTorch_to_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook to test the conversion of :

**PyTorch -.pth -->  ONNX -->  TensorflowRep object --> protobuf -.pb**


In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip uninstall -y pytorch
!pip uninstall -y torch

# you can just do 
#!pip install --upgrade --force-reinstall torch
# within a cell in Google Colab to get it working @Berenice

Uninstalling torch-1.1.0:
  Successfully uninstalled torch-1.1.0


In [0]:
torch_url=f"https://download.pytorch.org/whl/nightly/cpu/torch-1.2.0.dev20190804%2Bcpu-cp36-cp36m-linux_x86_64.whl"


!pip install -U {torch_url} torchvision

### PyTorch to ONNX

In [0]:

import torch.onnx
import torchvision


help(torch.onnx.export)

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
checkpoint_name = 'akash-mobilenet_v2-FER1-60perc.pt'
export_path = base_path + "ONNX/mobilenet_v2-256px.onnx"


# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.

# A model class instance (class not shown)
model = torchvision.models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(1280, 7)

#print(model)

# Load the weights from a file (.pth usually)
state_dict = torch.load(base_path + checkpoint_name, map_location=torch.device('cpu'))

# Load the weights now into a model net architecture defined by our class
model.load_state_dict(state_dict)

# Create the right input shape (e.g. for an image)
#dummy_input = torch.randn(1, 3, 224, 224)
dummy_input = torch.randn(1, 3, 256, 256)



torch.onnx.export(model, dummy_input, "onnx_model_name.onnx")

### Import ONNX to Tensorflow

In [0]:
!pip install onnx

In [0]:
!pip install onnx_tf

In [0]:
import onnx
import warnings
from onnx_tf.backend import prepare

warnings.filterwarnings('ignore') # Ignore all the warning messages in this tutorial

tf_model = onnx.load(export_path) # Load the ONNX file

tf_rep = prepare(tf_model) # Import the ONNX model to Tensorflow

In [0]:
# Now we have tf_rep, which is a python class containing four members: graph, inputs, outputs, and tensor_dict.

print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

['input.1']
-----
['465']
-----
{'classifier.1.bias': <tf.Tensor 'Const:0' shape=(1000,) dtype=float32>, 'classifier.1.weight': <tf.Tensor 'Const_1:0' shape=(1000, 1280) dtype=float32>, 'features.0.0.weight': <tf.Tensor 'Const_2:0' shape=(32, 3, 3, 3) dtype=float32>, 'features.0.1.bias': <tf.Tensor 'Const_3:0' shape=(32,) dtype=float32>, 'features.0.1.num_batches_tracked': <tf.Tensor 'Const_4:0' shape=() dtype=int64>, 'features.0.1.running_mean': <tf.Tensor 'Const_5:0' shape=(32,) dtype=float32>, 'features.0.1.running_var': <tf.Tensor 'Const_6:0' shape=(32,) dtype=float32>, 'features.0.1.weight': <tf.Tensor 'Const_7:0' shape=(32,) dtype=float32>, 'features.1.conv.0.0.weight': <tf.Tensor 'Const_8:0' shape=(32, 1, 3, 3) dtype=float32>, 'features.1.conv.0.1.bias': <tf.Tensor 'Const_9:0' shape=(32,) dtype=float32>, 'features.1.conv.0.1.num_batches_tracked': <tf.Tensor 'Const_10:0' shape=() dtype=int64>, 'features.1.conv.0.1.running_mean': <tf.Tensor 'Const_11:0' shape=(32,) dtype=float32>,

### Prepare an image for inference

In [0]:
!pip uninstall tensorflow-gpu
!pip install --upgrade tensorflow

In [0]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

print(tf.__version__)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

1.14.0


In [0]:

import numpy as np
from IPython.display import display
from PIL import Image

path_to_img = base_path + 'fer2013/PrivateTest/2/PrivateTest_1162474.jpg'
#path_to_img = base_path + 'fer2013/testface-happy.jpg'

img = Image.open(path_to_img).resize((256, 256))

new_img = np.broadcast_to(img, (1, 3, 256, 256)).copy()
print(new_img.shape)

display(new_img) # show the image

img_ycbcr = img.convert("YCbCr")
img_y, img_cb, img_cr = img_ycbcr.split()


#test_face = np.asarray(img_y, dtype=np.float32)[np.newaxis, np.newaxis, :, :]

In [0]:
# run the network
#dummy_face = torch.randn(1, 3, 224, 224)

print(new_img.shape)

test_face_run = tf_rep.run(new_img)._0

print(test_face_run.shape)

(1, 3, 256, 256)
(1, 1000)


### Tensorflow -.h5 to -.pb

In [0]:
#from tensorflow.python.saved_model import builder as pb_builder
import tensorflow as tf

base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
hdf5_name = 'dwiv-model.hdf5'

export_path_pb = base_path + "ONNX/convertedmodel.pb"


In [0]:
# load the model and save it as pb. 
# If the model is not in h5 format, convert it to h5 first.
#pre_model = tf.keras.models.load_model('path/to/your/model.model')
#pre_model.save('path/to/your/model.h5')


tf_rep.export_graph(export_path_pb) # export the TensorflowRep object to a -.pb file which can be imported to untiy.


### Inspect the -.pb file. Print out some node names

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
pb_path = base_path + 'ONNX/akash-mobilenet_v2-FER1-60perc.pb'
#pb_path = base_path + 'ONNX/mobilenet_v1_test.pb'
#pb_path = base_path + 'ONNX/weights_best.pb'

gf = tf.GraphDef()
gf.ParseFromString(open(pb_path,'rb').read())

15871857

In [0]:
#[n.name + '=>' +  n.op for n in gf.node if n.op in ( 'Softmax','Mul', 'Placeholder')]
[n.name + '=>' +  n.op for n in gf.node if n.op in ( 'Softmax','Mul', 'Placeholder', 'Reshape', 'Predictions', 'Add')]


['input.1=>Placeholder',
 'Reshape=>Reshape',
 'Reshape_1=>Reshape',
 'Reshape_2=>Reshape',
 'Reshape_3=>Reshape',
 'batchnorm/add=>Add',
 'batchnorm/mul=>Mul',
 'batchnorm/mul_1=>Mul',
 'batchnorm/mul_2=>Mul',
 'batchnorm/add_1=>Add',
 'Reshape_4=>Reshape',
 'Reshape_5=>Reshape',
 'Reshape_6=>Reshape',
 'Reshape_7=>Reshape',
 'batchnorm_1/add=>Add',
 'batchnorm_1/mul=>Mul',
 'batchnorm_1/mul_1=>Mul',
 'batchnorm_1/mul_2=>Mul',
 'batchnorm_1/add_1=>Add',
 'Reshape_8=>Reshape',
 'Reshape_9=>Reshape',
 'Reshape_10=>Reshape',
 'Reshape_11=>Reshape',
 'batchnorm_2/add=>Add',
 'batchnorm_2/mul=>Mul',
 'batchnorm_2/mul_1=>Mul',
 'batchnorm_2/mul_2=>Mul',
 'batchnorm_2/add_1=>Add',
 'Reshape_12=>Reshape',
 'Reshape_13=>Reshape',
 'Reshape_14=>Reshape',
 'Reshape_15=>Reshape',
 'batchnorm_3/add=>Add',
 'batchnorm_3/mul=>Mul',
 'batchnorm_3/mul_1=>Mul',
 'batchnorm_3/mul_2=>Mul',
 'batchnorm_3/add_1=>Add',
 'Reshape_16=>Reshape',
 'Reshape_17=>Reshape',
 'Reshape_18=>Reshape',
 'Reshape_19=>Res